**Importing the Libraries**

In [ ]:
from keras.preprocessing import sequence
from keras.models import Sequential, Model
from keras.layers import Dense, Embedding, GlobalMaxPooling1D, Flatten, Conv1D, Dropout, Activation, Input, concatenate
from keras.preprocessing.text import Tokenizer

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import os
import re
import string
!pip install emoji
import emoji
from emoji import UNICODE_EMOJI
from sklearn.model_selection import train_test_split

# For reproducibility
#from tensorflow import set_random_seed
from numpy.random import seed
seed(1)
tf.random.set_seed(2)

Using TensorFlow backend.


     |████████████████████████████████| 51kB 2.5MB/s 
  Created wheel for emoji: filename=emoji-0.5.4-cp36-none-any.whl size=42176 sha256=a737ee6cfc6d4451ca9908f4e7db876dfaa9ecbe213866c7f2413bbdaefa2fc5
  Stored in directory: /root/.cache/pip/wheels/2a/a9/0a/4f8e8cce8074232aba240caca3fade315bb49fac68808d1a9c
Successfully built emoji


**Setting Hyperparameters**

In [ ]:
# Hyperparams if GPU is available
if tf.test.is_gpu_available():
    # GPU
    BATCH_SIZE = 128 # Number of examples used in each iteration
    EPOCHS = 20 # Number of passes through entire dataset
    VOCAB_SIZE = 30000 # Size of vocabulary dictionary
    MAX_LEN = 50 # Max length of review (in words)
    EMBEDDING_DIM = 40 # Dimension of word embedding vector

# Hyperparams for CPU training
else:
    # CPU
    BATCH_SIZE = 32
    EPOCHS = 2
    VOCAB_SIZE = 20000
    MAX_LEN = 90
    EMBEDDING_DIM = 40

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


**Importing the dataset**

In [ ]:
#path = 'emotions1 - emotions1.csv'
path = 'Dataset_With_Emoji.csv'
df1 = pd.read_csv(path)

In [ ]:
tweets = (df1['Tweet'])
labels = (df1['Affect Dimension'])
df1.head()

,Unnamed: 0,Tweet,Affect Dimension,Unnamed: 2
0,0,pouting face @chetan_bhagat face with steam ...,anger,NaN
1,1,Low key Dro f*cked Molly like he had a point t...,anger,NaN
2,2,@TeleTransOne @eleganza_lily @FemFataleXX Tha...,anger,NaN
3,3,@F3Alpha #F3Roswell #Preblast\nI've face with...,anger,NaN
4,4,Did they offend us and they want it to sound n...,anger,NaN


**This can be uncommented in case the Dataset has emojis which we want to consider for classification**

In [ ]:
"""def is_emoji(s):
    count = 0
    for emoji in UNICODE_EMOJI:
        count += s.count(emoji)
        if count > 1:
            return False
    return bool(count)"""

'def is_emoji(s):\n    count = 0\n    for emoji in UNICODE_EMOJI:\n        count += s.count(emoji)\n        if count > 1:\n            return False\n    return bool(count)'

In [ ]:
"""def emoji2text(title):
    stemmed = []
    tokenized = []
    series = []
    print(type(series))
    for i in title:
      #print(type(title))
      for c in i:
        if is_emoji(c) == True:
          word1 = emoji.demojize(c)
          word1 = word1.replace('_', ' ')
          word1 = word1.replace(':', ' ')
          #word2 = " ".join(word1)
          i = i.replace(c, word1)
      series.append(i)
    return series"""

'def emoji2text(title):\n    stemmed = []\n    tokenized = []\n    series = []\n    print(type(series))\n    for i in title:\n      #print(type(title))\n      for c in i:\n        if is_emoji(c) == True:\n          word1 = emoji.demojize(c)\n          word1 = word1.replace(\'_\', \' \')\n          word1 = word1.replace(\':\', \' \')\n          #word2 = " ".join(word1)\n          i = i.replace(c, word1)\n      series.append(i)\n    return series'

In [ ]:
"""df1.Tweet = emoji2text(df1.Tweet)
tweets = (df1['Tweet'])
df1.head()"""

"df1.Tweet = emoji2text(df1.Tweet)\ntweets = (df1['Tweet'])\ndf1.head()"

**For One-Hot Encoding**

In [ ]:
encoded_labels = []
c= 0
for label in labels:
    if label == 'anger':
        encoded_labels.append(0)       
    elif label == 'joy':
        encoded_labels.append(1)       
    elif label == 'neutral':
        encoded_labels.append(2)       
    elif label == 'sadness/fear':
        encoded_labels.append(3)
df1['Affect Dimension']=np.asarray(encoded_labels)
#encoded_labels=np.asarray(encoded_labels)
#tweets1=np.array(tweets)
#print(encoded_labels.shape)
#print(tweets1.shape)

In [ ]:
df1['Affect Dimension'].unique()
joy = []
sadness = []
neutral = []
anger = []
for l in df1['Affect Dimension']:
    if l == 0:
        anger.append(1)
        joy.append(0)
        neutral.append(0)
        sadness.append(0)
    elif l == 1:
        anger.append(0)
        joy.append(1)
        neutral.append(0)
        sadness.append(0)
    elif l == 2:
        anger.append(0)
        joy.append(0)
        neutral.append(1)
        sadness.append(0)
    elif l == 3:
        anger.append(0)
        joy.append(0)
        neutral.append(0)
        sadness.append(1)

In [ ]:
df1['anger']= anger
df1['joy']= joy
df1['sadness'] = sadness
df1['neutral'] = neutral

In [ ]:
df1.head()

,Unnamed: 0,Tweet,Affect Dimension,Unnamed: 2,anger,joy,sadness,neutral
0,0,pouting face @chetan_bhagat face with steam ...,0,NaN,1,0,0,0
1,1,Low key Dro f*cked Molly like he had a point t...,0,NaN,1,0,0,0
2,2,@TeleTransOne @eleganza_lily @FemFataleXX Tha...,0,NaN,1,0,0,0
3,3,@F3Alpha #F3Roswell #Preblast\nI've face with...,0,NaN,1,0,0,0
4,4,Did they offend us and they want it to sound n...,0,NaN,1,0,0,0


**For splitting the Dataset into train and test**

In [ ]:
data_train, data_test = train_test_split(df1, test_size=0.2, random_state=42)

**For Data Preprocessing**

In [ ]:
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s): return re_tok.sub(r' \1 ', s).split()

In [ ]:
tokenizer = Tokenizer(num_words=VOCAB_SIZE)
tokenizer.fit_on_texts(data_train['Tweet'])

In [ ]:
x_train_seq = tokenizer.texts_to_sequences(data_train['Tweet'].values)
x_val_seq = tokenizer.texts_to_sequences(data_test['Tweet'].values)

x_train = sequence.pad_sequences(x_train_seq, maxlen=MAX_LEN, padding="post", value=0)
x_val = sequence.pad_sequences(x_val_seq, maxlen=MAX_LEN, padding="post", value=0)

#y_train, y_val = train['label'].values, val['label'].values

print('First sample before preprocessing: \n', data_train['Tweet'].values[0], '\n')
print('First sample after preprocessing: \n', x_train[0])

First sample before preprocessing: 
 I hate getting woken up out my sleep Ã°Å¸ËœÂ¡ face with steam from nose  

First sample after preprocessing: 
 [   3  213  199 3240   37   48   11  299  858    1    6   91   29   89
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0]


**Defining the parameters**

In [ ]:
NUM_FILTERS = 250
KERNEL_SIZE = 3
HIDDEN_DIMS = 250

In [ ]:
label_names = ['anger', 'joy', 'neutral', 'sadness']
y_train = data_train[label_names].values
y_test = data_test[label_names].values

**Creating the model**

In [ ]:
print('Build model...')
model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into EMBEDDING_DIM dimensions
model.add(Embedding(VOCAB_SIZE, EMBEDDING_DIM, input_length=MAX_LEN))
model.add(Dropout(0.2))

# we add a Convolution1D, which will learn NUM_FILTERS filters
model.add(Conv1D(NUM_FILTERS,
                 KERNEL_SIZE,
                 padding='valid',
                 activation='relu',
                 strides=1))

# we use max pooling:
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(HIDDEN_DIMS))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(4))
model.add(Activation('softmax'))
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Build model...
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 40)            1200000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 40)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 48, 250)           30250     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 250)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 250)               62750     
_________________________________________________________________
dropout_2 (Dropout)          (None, 250)               0         
_________________________________________________________________
activation_1 (Activation)    (None, 250

**Training the model**

In [ ]:
# fit a model
model.fit(x_train, y_train,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_split=0.2,
          verbose=2)

#pred = model.predict_classes(x_val)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 8205 samples, validate on 2052 samples
Epoch 1/20
 - 7s - loss: 1.2125 - accuracy: 0.4525 - val_loss: 0.7852 - val_accuracy: 0.7169
Epoch 2/20
 - 0s - loss: 0.5292 - accuracy: 0.8266 - val_loss: 0.4491 - val_accuracy: 0.8270
Epoch 3/20
 - 0s - loss: 0.2735 - accuracy: 0.9202 - val_loss: 0.4133 - val_accuracy: 0.8411
Epoch 4/20
 - 0s - loss: 0.1405 - accuracy: 0.9617 - val_loss: 0.4397 - val_accuracy: 0.8392
Epoch 5/20
 - 0s - loss: 0.0713 - accuracy: 0.9827 - val_loss: 0.4908 - val_accuracy: 0.8338
Epoch 6/20
 - 0s - loss: 0.0415 - accuracy: 0.9890 - val_loss: 0.5315 - val_accuracy: 0.8299
Epoch 7/20
 - 0s - loss: 0.0227 - accuracy: 0.9956 - val_loss: 0.5818 - val_accuracy: 0.8246
Epoch 8/20
 - 0s - loss: 0.0138 - accuracy: 0.9966 - val_loss: 0.6287 - val_accuracy: 0.8265
Epoch 9/20
 - 0s - loss: 0.0117 - accuracy: 0.9971 - val_loss: 0.6601 - val_accuracy: 0.8212
Epoch 10/20
 - 0s - loss: 0.0088 - accuracy: 0.9971 - val_loss: 0.6993 - val_accuracy: 0.8163
Epoch 11/20
 - 0s - l

**Testing the Model**

In [ ]:
# Evaluate the model
score, acc = model.evaluate(x_val, y_test, batch_size=BATCH_SIZE)
print('\nAccuracy: ', acc*100)

2565/2565 [==============================] - 0s 17us/step

Accuracy:  82.30019211769104
